In [7]:
#Imports
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.functional as F
import pandas as pd
import os
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
#Constants
img_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Images\Food Images"
data_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
batch_size = 128
learning_rate = 0.001
momentum = 0.9
img_size = 169
output_size = 13500

In [9]:

# constructing the dataloader
class foodImageDataset(Dataset):
    def __init__(self, img_dir, data_dir, transformation = None):
        self.img_dir = img_dir
        #self.ingr = pd.read_csv(data_dir, header=1,usecols=[5])
        self.transform = transformation
        self.labels = pd.read_csv(data_dir, header=1,usecols=[0,4])

    #Later augment the dataset to give more images per class to increase accuracy

    def __len__(self):
       return len(self.labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels.iloc[idx,1] + ".jpeg")
        try:
            image = read_image(img_path)
        except:
            return None
        label = self.labels.iloc[idx, 0]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __getingrs__(self, index):
        return self.ingr.iloc[index,0]

In [1]:
class Sampler(RandomSampler):
    #removes bad values
    
    def __init__(self, data_source, forbidden = []) -> None:
        super().__init__(data_source)
        self.data_source = data_source
        self.forbidden = forbidden
        self.refill()

    def remove(self, new_forbidden):
        # Remove numbers from the available indices
        for num in new_forbidden:
            if not (num in self.forbidden):
                self.forbidden.append(num)
        self._remove(new_forbidden)

    def _remove(self, to_remove):
        # Remove numbers just for this epoch
        for num in to_remove:
            if num in self.idx:
                self.idx.remove(num)

        self._num_samples = len(self.idx)

    def refill(self):
        # Refill the indices after iterating through the entire DataLoader
        self.idx = list(range(len(self.data_source)))
        self._remove(self.forbidden)

    def __iter__(self) -> Iterator[int]:
        for _ in range(self.num_samples // 32):
            batch = random.sample(self.idx, 32)
            self._remove(batch)
            yield from batch
        yield from random.sample(self.idx, self.num_samples % 32)
        self.refill()

NameError: name 'RandomSampler' is not defined

In [ ]:
#Model Declaration
class FoodModel(nn.Module):
    device = torch.cuda.set_device(0)

    def __init__(self,img_size, output_size):
        super().__init__()
        self.conv1 = nn.Conv2d(3,55,kernel_size=(9,9), stride=3, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
        print("init1")

        self.conv2 = nn.Conv2d(55,26,kernel_size=(5,5), stride=2, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))
        print("init2")

        self.flat = nn.Flatten()
        print("init3")

        self.fc3 = nn.Linear(4394, int(output_size*2))
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(.5)
        print("init4")
        
        self.fc4 = nn.Linear(output_size*2, output_size)
        print("init5")

    def forward(self, image):
        #First layer
        image = self.act1(self.conv1(image))
        image = self.drop1(image)
        print("train1")
        
        #Second layer
        image = self.act2(self.conv2(image))
        image = self.pool2(image)
        print("train2")

        #Third layer
        image = self.flat(image)
        image = self.act3(self.fc3(image))
        image = self.drop3(image)
        print("train3")

        #output
        image = self.fc4(image)
        return image
    
    def train(self, epochs, train_dl, loss_fn, opt):
        for epoch in range(epochs):
            count = 0
            for images, labels in train_dl:
                images = images.to(torch.device("cuda"))

                try:
                    preds = self(images)
                except:
                    print(labels)
                    continue
                loss = loss_fn(preds, labels)
                #training
                opt.zero_grad()
                loss.backward()
                opt.step()
                count += 1

SyntaxError: invalid syntax (1686043050.py, line 53)

In [ ]:
model = FoodModel(img_size, output_size)

init1
init2
init3


init4
init5


In [ ]:
#sampler
sampler = Sampler()

In [ ]:
#Setting Model
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((169,169)),
    transforms.ToTensor(),
])
food = foodImageDataset(img_dir, data_dir, transformation=transform)
food_dl = DataLoader(food, batch_size, sampler=)

In [ ]:
#Running Model
model.train(5,food_dl,loss_fn,opt)

NameError: name 'model' is not defined

In [ ]:
def get(img_dir, data_dir, transformation = None):
        data = pd.read_csv(data_dir, header=1)
        
        # Extracting specific columns for labels and ingredients
        labels = data.iloc[:, [1,4]]  # Assuming the label is in column 1 and ingredient in column 4
        print(labels)
        ingr = data.iloc[:, 5]
        print(ingr[6957, 1])

get(img_dir,data_dir)